### Import package

In [1]:
#LiveLattice
from livelattice.processing import *
from livelattice.pipeline import *
import sys
from PyQt5.QtWidgets import QFileDialog, QApplication

%load_ext autoreload
%autoreload 2
#CAMDU
import tifffile
import napari

rmm not available


E:\LauraCooper\livelattice-env\lib\site-packages\pycudadecon\_libwrap.py:45: UserWarning: Unable to find function: camcor_interface_init in libcudaDecon
  def camcor_interface_init(  # type: ignore [empty-body]
E:\LauraCooper\livelattice-env\lib\site-packages\pycudadecon\_libwrap.py:52: UserWarning: Unable to find function: camcor_interface in libcudaDecon
  def camcor_interface(  # type: ignore [empty-body]


### Select the folders to process using automatic PSF detection

In [111]:
#number_of_dates = 1

# Linux
#root_path = '/run/user/1000/gvfs/smb-share:server=taurus0.jslab.ucsd.edu,share=data/MOSAIC_Data/Data/'
#output_path = '/run/user/1000/gvfs/smb-share:server=taurus0.jslab.ucsd.edu,share=processing/MOSAIC_Data/Processed_Data/'

#app = QApplication(sys.argv)
#all_data_path = []
#for index in range(number_of_dates):
#    data_path = QFileDialog.getExistingDirectory(None, "Select data directory #"+str(index+1)+" ...", root_path).split('/')[-1]
#    all_data_path.append(data_path)

#tif_path_list, save_path_list = collect_data_locations(root_path, all_data_path, output_path)

#specified_channels = None # None for auto detection, use a list of wave numbers (string) if to manually specify, e.g. ['488', '642']

#selected_psf_path = None # None for auto detection, use a list of PSF paths if to manually specify

#del app
image_path='E:\LauraCooper\MWA\Raw-data-example-5D.sld - Raw_data_slide3_capture12-1.tif'
image_path='E:\LauraCooper\MWA\Raw-data-example-5D.sld - Raw_data_slide3_capture12-1 every4thslice.tif'
image=tifffile.imread(image_path)
psf_path='E:\LauraCooper\MWA\PSFs-as-tiffs\\2024-09-13_PSF_488.tif'
psf_stack=tifffile.imread(psf_path)
viewer = napari.Viewer()
viewer.add_image(image, name='image', colormap='gray')
napari.run()

### Set parameters and process the movies

In [112]:
# select specific folders and/or frames within each folder to process
# use None if to process all
# index starts with 0
folders_to_process = None
frames_to_process = None

# options
save_mip = True # whether to create the MIP folder with individual and combined MIP images
overwrite_file = True # whether to overwrite exisiting files, use False if continue previous run
use_dask = False

# select operations to perform
num_decon_it = 15
decon_zsection_size = 400 # to avoid memory overflow, may lower if deconvolution is slow
decon_deskew_rotate = True
no_decon = False
decon_only = False

# run the processing
#process_data(tif_path_list, save_path_list,
#             specified_channels, selected_psf_path,
#             folders_to_process, frames_to_process,
#             save_mip, overwrite_file, use_dask,
#             num_decon_it, decon_zsection_size, 
#             decon_deskew_rotate, no_decon, decon_only)
processing = image

In [113]:
t1 = time.time()
num_zsections = 0
if image.shape[0] % decon_zsection_size == 0:
    num_zsections = int(image.shape[0] / decon_zsection_size)
else:
    num_zsections = int(image.shape[0] / decon_zsection_size) + 1

pad_size = int(0.1*decon_zsection_size)
start_z = pad_size
full_deconv = None

for ind_z in range(num_zsections):
      
    section = processing[start_z-pad_size:start_z+decon_zsection_size+pad_size]
    if section.shape[0] < pad_size:
        break
                
    print('\nDeconvolution starts for z-section', ind_z+1)
    with TemporaryOTF(psf=psf_path, dzpsf=dzpsf, dxpsf=dxpsf, fixorigin=0, otf_bgrd=0, max_otf_size=None, skewed_decon=True) as otf:
        with RLContext(section.shape, otf.path, dzdata, dxdata, dzpsf, dxpsf, skewed_decon=True) as ctx:
                
            section_deconv = rl_decon(section, background=0, n_iters=num_decon_it, skewed_decon=True)

            # to avoid boundary effect due to partitioning in all x,y,z, we divide only z in blocks for large dataset
            if start_z == pad_size:
                full_deconv = section_deconv[:pad_size+decon_zsection_size]
            else:
                full_deconv = np.concatenate([full_deconv, section_deconv[pad_size:pad_size+decon_zsection_size]])

            start_z += decon_zsection_size
                    
    #processing = full_deconv.astype('uint16')

In [114]:
viewer = napari.Viewer()
viewer.add_image(full_deconv, name='deconv', colormap='gray')
napari.run()

TypeError: 'NoneType' object is not subscriptable

In [119]:
angle=32.8
dzdata=0.5*4
dxdata=0.104

print(image.shape)
processing = cupy.asarray(image)
#Swap direction of z axis Needed for 3i data
processing = cupy.flip(processing, 0)
print(processing.shape)

# Step 1
swapped = cupy.swapaxes(processing, 0, 2)
print(swapped.shape)

rotated = cupy.flip(swapped, 2)

# Padded not needed
# Pad to avoid cropping during transform
#angle_rad = angle * np.pi / 180.0
#pad_z = int(rotated.shape[0] * (1 - np.sin(angle_rad)))
#pad_width = [(pad_z, pad_z), (0, 0), (0, 0)]
#padded = np.pad(rotated, pad_width, mode='constant', constant_values=0)
#print('padded',padded.shape)
#padded=rotated
#print('notpadded:',padded.shape)
# Step 2
transformed = scale_and_shear_cupy(rotated, angle, dzdata, dxdata)
print(transformed.shape)

# move to CPU memory
deskew = cupy.asnumpy(transformed)
print(deskew.shape)

(15, 263, 326)
(15, 263, 326)
(326, 263, 15)
(178, 264, 544)
(178, 264, 544)


In [120]:
def crop_to_nonzero(data):
    # Find indices of non-zero elements
    nonzero = np.nonzero(data)

    # Get bounding box
    zmin, ymin, xmin = np.min(nonzero, axis=1)
    zmax, ymax, xmax = np.max(nonzero, axis=1) + 1  # +1 to include endpoint

    # Crop the array
    cropped = data[zmin:zmax, ymin:ymax, xmin:xmax]
    return cropped

In [121]:
cropped = crop_to_nonzero(deskew)
print(cropped.shape)  # Should be (4, 6, 8)

(177, 263, 543)


In [122]:
viewer = napari.Viewer()
#viewer.add_image(image, name='image', colormap='gray')
viewer.add_image(cupy.asnumpy(cropped), name='cropped padded', colormap='gray')
napari.run()